In [14]:
#Lettura del file assets_dataset.csv
File_Assets=open("assets_dataset.csv", "r")
File_Cov=open("./assets_cov_matrix.csv", "r")
vet = {}
for count, i in enumerate(File_Assets):
    if count!=0:
        id=i.split("_")[1].split(",")[0]
        num1=i.split(",")[1].strip()
        num2=i.split(",")[2].strip()
        vet[int(id)]=[float(num1),float(num2)]

In [15]:
#Lettura del file assets_cov_matrix.csv
MatCov=[]
n=[]
for count, i in enumerate(File_Cov):
    if count!=0:
        n=i.split(",")
        for j in range(500-1):
            n[j]=float(n[j+1])
        MatCov.append(n)

In [10]:
import qubovert as QbVrt

vars = [QbVrt.boolean_var(f'x{j}') for j in range(10)]
model = QbVrt.QUBO()
solution = QbVrt.anneal_qubo(model,  num_anneals=10)




0,0,0,1,0

AttributeError: module 'qubovert' has no attribute 'anneal_qubo'